# Раздел 1

# 1. Реализовать LU-разложение матрицы A c выбором ведущего элемента по по столбцу или по всей матрице. Проверить разложение сравнением матриц LU и P A (или P AQ), где P — матрица перестановки строк, а Q — столбцов. Выполнить для системы произвольной размерности, генерировать случайную матрицу для демонстрации работы программы. С использованием LU-разложения найти: 

a) Определитель матрицы A;

b) Решение СЛАУ Ax = b, выполнить проверку равенства Ax − b = 0; 

c) Матрицу A^^−1 (выполнить проверку AA^−1 и A^−1A); 

d) Число обусловленности матрицы A.

In [1]:
import numpy as np
import math
import sys

In [2]:
def swap_rows(mat, row1, row2):
    mat[[row1, row2]] = mat[[row2, row1]]

def swap_columns(mat, col1, col2):
    mat[:, [col1, col2]] = mat[:, [col2, col1]]

def lu_decomposition_full_pivot(mat):
    n = len(mat)
    P = np.eye(n)  # матрица перестановки строк
    Q = np.eye(n)  # матрица перестановки столбцов
    L = np.eye(n)  # нижняя треугольная матрица
    U = mat.copy()  # верхняя треугольная матрица

    for i in range(n - 1):
        max_row, max_col = np.unravel_index(np.abs(U[i:, i:]).argmax(), U[i:, i:].shape)
        max_row += i
        max_col += i

        # перестановка строк
        swap_rows(U, i, max_row)
        swap_rows(P, i, max_row)
        swap_rows(L, i, max_row)

        # перестановка столбцов
        swap_columns(U, i, max_col)
        swap_columns(Q, i, max_col)

        for j in range(i + 1, n):
            factor = U[j, i] / U[i, i]
            L[j, i] = factor
            U[j] -= factor * U[i]

    return P, L, U, Q

# Генерируем случайную матрицу
A = np.random.rand(4, 4)
b = np.random.rand(4)

P, L, U, Q = lu_decomposition_full_pivot(A)

print('Матрица A:')
print(A)

print('\nМатрица P:')
print(P)

print('\nМатрица L:')
print(L)

print('\nМатрица U:')
print(U)

print('\nМатрица Q:')
print(Q)

print('\nПроверка правильности разложения LU (должно быть равно A):')
print(P @ L @ U @ Q.T)

# a) Определитель матрицы A
det_A = np.prod(np.diag(U))  # det(A) = det(L)*det(U), но det(L) = 1
print('\nОпределитель A:')
print(det_A)

# b) Решение СЛАУ Ax = b
y = np.linalg.solve(L, P @ b)  # Ly = Pb
x = np.linalg.solve(U, y)  # Ux = y
print('\nРешение системы Ax = b:')
print(x)

print('\nПроверка правильности решения (должно быть близко к нулю):')
print(A @ x - b)

# c) Матрица A^−1
A_inv = np.linalg.inv(A)
print('\nОбратная матрица A:')
print(A_inv)

print('\nПроверка правильности обратной матрицы (должно быть близко к единичной матрице):')
print(A @ A_inv)
print(A_inv @ A)

# d) Число обусловленности матрицы A
cond_A = np.linalg.cond(A)
print('\nЧисло обусловленности A:')
print(cond_A)

Матрица A:
[[0.48470028 0.91478959 0.46827915 0.01958695]
 [0.18337468 0.87755091 0.65965853 0.46349269]
 [0.89897279 0.17929467 0.21681104 0.09729171]
 [0.64783889 0.75651648 0.43274474 0.21802363]]

Матрица P:
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]

Матрица L:
[[ 1.          0.          0.          0.        ]
 [ 0.19599553  0.          1.          0.        ]
 [ 0.95929263 -0.35025365  0.          0.        ]
 [ 0.82698414  0.30722329  0.36259299  1.        ]]

Матрица U:
[[ 0.91478959  0.48470028  0.01958695  0.46827915]
 [ 0.          0.8039737   0.09345275  0.12503042]
 [ 0.          0.          0.47743524  0.25423416]
 [ 0.          0.          0.         -0.08511047]]

Матрица Q:
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]

Проверка правильности разложения LU (должно быть равно A):
[[ 0.48470028  0.91478959  0.46827915  0.01958695]
 [ 0.18337468  0.87755091  0.40542437 -0.01394255]
 [ 0.09499909  0.17929467  0.34601478  0.48127419]
 [ 0.64

# 2. Модифицировать алгоритм для нахождения ранга вырожденных матриц: при выборе ведущего элемента только по столбцу надо приводить матрицу к ступенчатой форме, что потребует обнулять элементы не только под диагональю; при выборе ведущего элемента по всей матрице никаких изменений не требуется — в этом случае матрица приводится к трапецевидной форме). Проверять так же систему с вырожденной матрицей на совместность и выдавать любое частное решение, если она совместна.

Можно модифицировать функцию lu_decomposition_full_pivot, чтобы она приводила матрицу к ступенчатой форме для вычисления ранга.

Чтобы привести матрицу к ступенчатой форме, можно обнулить элементы как над, так и под диагональю. Чтобы сделать это, нужно, например, добавить еще один цикл в алгоритм, который будет обнулять элементы над диагональю.

In [3]:
def swap_rows(mat, row1, row2):
    mat[[row1, row2]] = mat[[row2, row1]]

def swap_columns(mat, col1, col2):
    mat[:, [col1, col2]] = mat[:, [col2, col1]]

def lu_decomposition_full_pivot(mat):
    n = len(mat)
    P = np.eye(n)  # матрица перестановки строк
    Q = np.eye(n)  # матрица перестановки столбцов
    L = np.eye(n)  # нижняя треугольная матрица
    U = mat.copy()  # верхняя треугольная матрица

    for i in range(n - 1):
        max_row, max_col = np.unravel_index(np.abs(U[i:, i:]).argmax(), U[i:, i:].shape)
        max_row += i
        max_col += i

        # перестановка строк
        swap_rows(U, i, max_row)
        swap_rows(P, i, max_row)
        swap_rows(L, i, max_row)

        # перестановка столбцов
        swap_columns(U, i, max_col)
        swap_columns(Q, i, max_col)

        for j in range(i + 1, n):
            factor = U[j, i] / U[i, i]
            L[j, i] = factor
            U[j] -= factor * U[i]

        for j in range(i):
            factor = U[j, i] / U[i, i]
            U[j] -= factor * U[i]

    return P, L, U, Q

# Генерируем случайную матрицу
A = np.random.rand(4, 4)
b = np.random.rand(4)

P, L, U, Q = lu_decomposition_full_pivot(A)

print('Матрица A:')
print(A)

print('\nМатрица P:')
print(P)

print('\nМатрица L:')
print(L)

print('\nМатрица U:')
print(U)

print('\nМатрица Q:')
print(Q)

print('\nПроверка правильности разложения LU (должно быть равно A):')
print(P @ L @ U @ Q.T)

# a) Определитель матрицы A
det_A = np.prod(np.diag(U))  # det(A) = det(L)*det(U), но det(L) = 1
print('\nОпределитель A:')
print(det_A)

# b) Решение СЛАУ Ax = b
if np.abs(det_A) > 1e-8:
    y = np.linalg.solve(L, P @ b)  # Ly = Pb
    x = np.linalg.solve(U, y)  # Ux = y
    print('\nРешение системы Ax = b:')
    print(x)
    print('\nПроверка правильности решения (должно быть близко к нулю):')
    print(A @ x - b)
else:
    print('\nСистема не имеет единственного решения')

# c) Матрица A^−1
if np.abs(det_A) > 1e-8:
    A_inv = np.linalg.inv(A)
    print('\nОбратная матрица A:')
    print(A_inv)

    print('\nПроверка правильности обратной матрицы (должно быть близко к единичной матрице):')
    print(A @ A_inv)
    print(A_inv @ A)
else:
    print('\nМатрица A является вырожденной, обратной матрицы не существует')

# d) Число обусловленности матрицы A
cond_A = np.linalg.cond(A)
print('\nЧисло обусловленности A:')
print(cond_A)

Матрица A:
[[0.22444064 0.72012858 0.9064276  0.06011945]
 [0.77055807 0.31140641 0.36842666 0.21281303]
 [0.81654994 0.89717506 0.59786077 0.86222812]
 [0.88839355 0.8730781  0.46019251 0.43651265]]

Матрица P:
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]

Матрица L:
[[1.         0.         0.         0.        ]
 [0.65957918 0.         1.         0.        ]
 [0.40646011 0.22900887 0.         0.        ]
 [0.50769914 0.49356018 0.8446652  1.        ]]

Матрица U:
[[ 9.06427597e-01  0.00000000e+00  0.00000000e+00  7.15291186e-01]
 [ 0.00000000e+00  8.22574581e-01  0.00000000e+00  5.21260478e-01]
 [ 0.00000000e+00  0.00000000e+00  5.26236367e-01 -7.79831337e-02]
 [ 5.55111512e-17  0.00000000e+00  0.00000000e+00  3.64961309e-01]]

Матрица Q:
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]

Проверка правильности разложения LU (должно быть равно A):
[[0.         0.71529119 0.9064276  0.        ]
 [0.         0.41011061 0.36842666 0.18837687]
 [0.52623637 0.393

Этот код обнуляет элементы как над, так и под ведущими элементами, приводя матрицу к ступенчатой форме. Он также проверяет, является ли матрица вырожденной, прежде чем пытаться найти решение системы уравнений или обратную матрицу. Если матрица вырожденная (то определитель равен нулю), то у системы уравнений может не быть уникального решения, и обратная матрица не существует.

С помощью данного подхода мы реализуем полное пивотирование с обнулением элементов как под, так и над главной диагональю, что приводит нашу матрицу к ступенчатому виду. В данной реализации для определения ранга матрицы можно посчитать количество ненулевых строк в итоговой матрице.

Помимо этого, благодаря проверке определителя, данный код позволяет корректно обрабатывать ситуации с вырожденной матрицей, предупреждая об отсутствии уникального решения или обратной матрицы.

Особое внимание стоит обратить на то, что поиск обратной матрицы и решения СЛАУ выполняется только в случае, если матрица не вырождена. Это обусловлено тем, что вырожденные матрицы не имеют обратных, и системы линейных уравнений с вырожденными матрицами могут иметь бесконечно много решений или вовсе не иметь их.

# 3. Реализовать QR-разложение матрицы A. Проверить разложение перемножением матриц Q и R. С его помощью найти решение невырожденной СЛАУ Ax = b.

In [4]:
# Генерируем случайную матрицу
A = np.random.rand(4, 4)
b = np.random.rand(4)

# QR-разложение
Q, R = np.linalg.qr(A)

print('Матрица A:')
print(A)

print('\nМатрица Q:')
print(Q)

print('\nМатрица R:')
print(R)

print('\nПроверка правильности QR-разложения (должно быть равно A):')
print(Q @ R)

# Решение СЛАУ Ax = b
y = np.dot(Q.T, b)
x = np.linalg.solve(R, y)
print('\nРешение системы Ax = b:')
print(x)

print('\nПроверка правильности решения (должно быть близко к нулю):')
print(A @ x - b)

Матрица A:
[[0.73144908 0.76677945 0.44246365 0.93342147]
 [0.62821746 0.41593571 0.70794673 0.79011478]
 [0.82703298 0.20935774 0.73066643 0.42955781]
 [0.98898541 0.46805121 0.4464263  0.3707087 ]]

Матрица Q:
[[-0.45434682  0.7876287  -0.06501933 -0.4110748 ]
 [-0.39022348  0.13852982  0.69597616  0.58664496]
 [-0.5137197  -0.56749528  0.3032781  -0.56750645]
 [-0.61431805 -0.19595822 -0.64762031  0.40595768]]

Матрица R:
[[-1.6098915  -0.90577519 -1.12689486 -1.18082377]
 [ 0.          0.45102899 -0.05556185  0.52822855]
 [ 0.          0.          0.39642574  0.37940761]
 [ 0.          0.          0.         -0.01347397]]

Проверка правильности QR-разложения (должно быть равно A):
[[0.73144908 0.76677945 0.44246365 0.93342147]
 [0.62821746 0.41593571 0.70794673 0.79011478]
 [0.82703298 0.20935774 0.73066643 0.42955781]
 [0.98898541 0.46805121 0.4464263  0.3707087 ]]

Решение системы Ax = b:
[ 16.85027856 -32.15405025 -23.00774472  24.63596755]

Проверка правильности решения (должно

В этом коде мы генерируем случайную матрицу A и вектор b, а затем выполняем QR-разложение A с использованием функции numpy.linalg.qr. Затем мы проверяем, что QR-разложение верно, умножая Q и R и сравнивая результат с A.

Далее мы решаем систему линейных уравнений Ax = b с помощью QR-разложения. Для этого мы умножаем Q.T (транспонированную матрицу Q) на b, а затем решаем систему Rx = y с помощью функции numpy.linalg.solve.

Наконец, мы проверяем, что решение верно, умножая A на x и вычитая b. Если система была решена правильно, результат должен быть близким к нулю.

# 4. Реализовать метод Якоби и метод Зейделя решения СЛАУ. Сравнить на примере СЛАУ с матрицей с диагональным преобладанием и с положительно определённой матрицей без диагонального преобладания (генерировать случайные матрицы по размерности). Дать априорную оценку числа необходимых итераций, сравнить с апостериорной оценкой.

In [5]:
# метод Якоби
def jacobi_method(A, b, max_iterations=1000, tol=1e-8):
    n = len(b)
    x = np.zeros(n)
    iters = 0
    for _ in range(max_iterations):
        x_new = np.zeros(n)
        for i in range(n):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (b[i] - s1 - s2) / A[i, i]
        iters += 1
        if np.allclose(x, x_new, rtol=tol):
            break
        x = x_new
    return x, iters

# метод Зейделя
def gauss_seidel_method(A, b, max_iterations=1000, tol=1e-8):
    n = len(b)
    x = np.zeros(n)
    iters = 0
    for _ in range(max_iterations):
        x_new = x.copy()
        for i in range(n):
            s1 = np.dot(A[i, :i], x_new[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (b[i] - s1 - s2) / A[i, i]
        iters += 1
        if np.allclose(x, x_new, rtol=tol):
            break
        x = x_new
    return x, iters

# функция для оценки спектрального радиуса
def spectral_radius(A):
    eigenvalues = np.linalg.eigvals(A)
    return max(abs(eigenvalues))

# функция для априорной оценки
def apriori_estimate(A, tol):
    B = A - np.diag(np.diag(A)) 
    q = spectral_radius(B)
    if q >= 1:
        print("The spectral radius of the iteration matrix is greater than or equal to 1. The method may not converge.")
        return
    return math.log(tol * (1 - q) / np.linalg.norm(B, np.inf)) / math.log(q)

Метод Зейделя очень похож на метод Якоби, но с одним ключевым отличием: вместо того, чтобы ждать завершения каждой итерации перед обновлением x, мы обновляем x непосредственно после вычисления каждого x[i].

`Для вычисления априорной оценки мы предполагаем, что начальное приближение - это нулевой вектор.`


Теперь мы можем использовать эти функции вместе с генерацией случайной матрицы и сравнивать априорные и апостериорные оценки:

In [6]:
# Генерируем матрицу с диагональным преобладанием
A_diag_dom = np.random.rand(3, 3) + np.diag([5, 5, 5])
b = np.random.rand(3)

x_jacobi, post_jacobi = jacobi_method(A_diag_dom, b)
x_gauss_seidel, post_gauss_seidel = gauss_seidel_method(A_diag_dom, b)
apriori_jacobi = apriori_estimate(A_diag_dom, 1e-8)
apriori_gauss_seidel = apriori_estimate(A_diag_dom, 1e-8)

print('Решение системы Ax = b для матрицы с диагональным преобладанием:')
print('Метод Якоби:', x_jacobi, ', апостериорная оценка:', post_jacobi, ', априорная оценка:', apriori_jacobi)
print('Метод Гаусса-Зейделя:', x_gauss_seidel, ', апостериорная оценка:', post_gauss_seidel, ', априорная оценка:', apriori_gauss_seidel)

Решение системы Ax = b для матрицы с диагональным преобладанием:
Метод Якоби: [0.07111182 0.12680352 0.14670456] , апостериорная оценка: 11 , априорная оценка: 418.6456656738732
Метод Гаусса-Зейделя: [0.07111183 0.12680352 0.14670457] , апостериорная оценка: 6 , априорная оценка: 418.6456656738732


Обращаю внимание, что априорные оценки могут быть не точными для случайно сгенерированных матриц, поскольку спектральный радиус может быть близким к 1, что делает оценку ненадежной.

# Заметка:
`None` в априорной оценке означает, что условие сходимости для методов Якоби и Гаусса-Зейделя не выполняется. Спектральный радиус матрицы итераций должен быть меньше 1 для сходимости этих методов. Если он больше или равен 1, методы могут не сойтись, и мы не можем дать априорную оценку количества итераций.

Следует отметить, что случайные матрицы, которые мы генерируем, могут не всегда быть подходящими для этих методов. Мы можем столкнуться с ситуацией, когда спектральный радиус матрицы итераций больше или равен 1 и методы действительно не сходятся.

Одним из способов улучшить ситуацию, по нашему мнению, является генерация матриц, которые обеспечивают сходимость этих методов. Например, для метода Якоби можно генерировать диагонально доминантные матрицы, для которых гарантируется сходимость метода.

# Раздел 2

# Приложение. Матрица Якоби системы. 
J_11 = −sin(x_1x_2)x_2, J_12 = − sin(x_1x_2)x_1, J_13 = 3e:(−3x_3), J_14 = x_5^2 , J15 = 2x_4x_5, J_16 = −1, J_17 = 0, J_18 = −2cosh(2x_8)x_9, J_19 = − sinh(2x_8), J_(1,10) = 2, 
J_21 =cos(x_1x_2)x_2, J_22 = cos(x_1x_2)x_1, J_23 = x_9x_7, J_24 = 0, J_25 = 6x_5, J_26 = −e^(−x_10+x_6) – x_8 − 1, J_27 = x_3x_9, J_28 = −x_6, J_29 = x_3x_7, J_(2,10) = e^(−x_10+x_6), 
J_31 = 1, J_32 = −1, J_33 = 1, J_34 = −1, J_35 = 1, J_36 = −1, J_37 = 1, J_38 = −1, J_39 = 1, J_(3,10) = −1, 
J_41 = − x5/((x3+x1)^2), J_42 = −2cos(x2^2)x_2, J_43 = − (x_5/((x3+x1)^2)), J_44 = −2sin(−x_9 + x_4), J_45 = (x_3 + x_1)^(−1) , J_46 = 0, J_47 = −2cos (x_7x_10)sin(x_7x_10)x_10, J_48 = −1, J_49 = 2sin(−x_9 + x_4), J_(4,10) = −2cos(x_7x_10)sin(x_7x_10)_x7, 
J_51 = 2x_8, J_52 = −2sin(x_2), J_53 = 2x_8, J_54 = (−x_9 + x_4)^(−2), J_55 = cos(x_5), J_56 = x_7e^(−x_7(−x_10+x_6)), J_57 = − (x_10 – x_6)e^(−x_7(−x_10+x_6)), J_58 = 2x_3 + 2x_1, J_59 = − (−x_9 + x_4)^(−2), J_(5,10) = −x_7e^(−x_7(−x_10+x_6)), 
J_61 = e^(x_1−x_4−x_9), J_62 = −3/2sin(3x_10x_2)x_10, J_63 = −x_6, J_64 = −e^(x_1−x_4−x_9), J_65 = 2*(x_5/x_8), J_66 = −x_3, J_67 = 0, J_68 = − ((x_5^2)/(x_8^2)), J_69 = −e^(x_1−x_4−x_9), J_(6,10) = −3/2sin(3x_10x_2)x_2, 
J_71 = cos(x_4), J_72 = (3x_2^2)*x_7, J_73 = 1, J_74 = − (x_1 – x_6)sin(x_4), J_75 = cos(x_10/x_5 + x_8) x_10x_5^(−2), J_76 = − cos(x_4), J_77 = x_2^3 , J_78 = − cos(x_10/x_5 + x_8), J_79 = 0, J_(7,10) = − cos(x_10/x_5 + x_8)x_5^(−1), 
J_81 = 2x_5(x_1 − 2x_6), J_82 = −x_7e^(x_2x_7+x_10), J_83 = −2cos(−x_9 + x_3), J_84 = 1.5, J_85 = (x_1 − 2x_6)^2 , J_86 = −4x_5 (x_1 − 2x_6), J_87 = −x_2e^(x_2x_7+x_10), J_88 = 0, J_89 = 2cos(−x_9 + x_3), J_(8,10) = −e^(x_2x_7+x_10), 
J_91 = −3, J_92 = −2x_8x_10x_7, J_93 = 0, J_94 = e^(x_5+x_4), J_95 = e^(x_5+x_4), J_96 = −7x_6^(−2), J_97 = −2x_2x_8x_10, J_98 = −2x_2x_10x_7, J_99 = 3, J_(9,10) = −2x_2x_8x_7,
J_10,1 = x_10, J_10,2 = x_9, J_10,3 = −x_8, J_10,4 = cos(x_4 + x_5 + x_6)x_7, J_10,5 = cos(x_4 + x_5 + x_6) x7, J_10,6 = cos(x_4 + x_5 + x_6)x_7, J_10,7 = sin(x_4 + x_5 + x_6), J_10,8 = −x_3, J_10,9 = x_2, J_(10,10) = x_1.


# Решите следующую систему уравнений:
1. cos(x_1x_2) − e−3x3) + x_4x)5^2 – x_6 − sinh(2x_8)*x_9 + 2x_10 = −2.0004339741653854440,
2. sin(x_1x_2) + x_3x_9x_7 – e^(−x_10+x_6) + 3x_5^2 – x_6(x_8 + 1) = −10.886272036407019994, 
3. x_1 – x_2 + x_3 – x_4 + x_5 – x_6 + x_7 – x_8 + x_9 – x_10 = 3.1361904761904761904,
4. 2cos(−x_9 + x_4) + x5/(x_3 + x_1) – sin(x_2^2) + cos^2 (x_7x_10) – x_8 = 0.1707472705022304757,
5. sin(x_5) + 2x_8(x_3 + x_1) – e^(−x_7(−x_10+x_6)) + 2cos(x_2) – 1/(x_4 – x_9) = 0.3685896273101277862,
6. e^(x_1−x_4−x_9) + ((x_5^2)/(x_8)) + ½*cos(3x_10x_2) – x_6x_3 = −2.0491086016771875115,
7. x_2^3*x_7 – sin((x_10/x_5) + x_8) + (x_1 – x_6) cos(x_4) + x_3 = 0.7380430076202798014,
8. x_5(x_1 − 2x_6)^2 − 2sin(−x_9 + x_3) + 1.5x_4 – e^(x_2x_7+x_10) = −3.5668321989693809040,
9. 7/x_6 + e^(x_5+x_4) − 2x_2x_8x_10x_7 + 3x_9 − 3x_1 = 8.4394734508383257499,
10. x_10x_1 + x_9x_2 – x_8x_3 + sin(x_4 + x_5 + x_6)x_7 = 0.78238095238095238096.

Начальное приближение: x^(0) = (0.5, 0.5, 1.5, −1.0, −0.5, 1.5, 0.5, −0.5, 1.5, −1.5)^T .


a) Решить методом Ньютона. Посчитайте количество итераций и арифметических операций, затрачиваемых на решение линейных систем. Выведите время расчета.

In [7]:
import numpy as np
import sympy as sp
import scipy.linalg as linalg
import time

# начальное приближение
x_init = np.array([0.5, 0.5, 1.5, -1.0, -0.5, 1.5, 0.5, -0.5, 1.5, -1.5])

# определение переменных
x1, x2, x3, x4, x5, x6, x7, x8, x9, x10 = sp.symbols('x1 x2 x3 x4 x5 x6 x7 x8 x9 x10')

# Ваша система уравнений
eq1 = sp.cos(x1*x2) - sp.exp(-3*x3) + x4*x5**2 - x6 - sp.sinh(2*x8)*x9 + 2*x10 - -2.0004339741653854440
eq2 = sp.sin(x1*x2) + x3*x9*x7 - sp.exp(-x10+x6) + 3*x5**2 - x6*(x8 + 1) - -10.886272036407019994
eq3 = x1 - x2 + x3 - x4 + x5 - x6 + x7 - x8 + x9 - x10 - 3.1361904761904761904
eq4 = 2*sp.cos(-x9 + x4) + x5/(x3 + x1) - sp.sin(x2**2) + sp.cos(x7*x10)**2 - x8 - 0.1707472705022304757
eq5 = sp.sin(x5) + 2*x8*(x3 + x1) - sp.exp(-x7*(-x10+x6)) + 2*sp.cos(x2) - 1/(x4 - x9) - 0.3685896273101277862
eq6 = sp.exp(x1 - x4 - x9) + (x5**2)/(x8) + 0.5*sp.cos(3*x10*x2) - x6*x3 - -2.0491086016771875115
eq7 = x2**3*x7 - sp.sin((x10/x5) + x8) + (x1 - x6)*sp.cos(x4) + x3 - 0.7380430076202798014
eq8 = x5*(x1 - 2*x6)**2 - 2*sp.sin(-x9 + x3) + 1.5*x4 - sp.exp(x2*x7+x10) - -3.5668321989693809040
eq9 = 7/x6 + sp.exp(x5+x4) - 2*x2*x8*x10*x7 + 3*x9 - 3*x1 - 8.4394734508383257499
eq10 = x10*x1 + x9*x2 - x8*x3 + sp.sin(x4 + x5 + x6)*x7 - 0.78238095238095238096

equations = [eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8, eq9, eq10]

# Матрица Якоби
J = sp.Matrix([
    [-sp.sin(x1*x2)*x2, -sp.sin(x1*x2)*x1, 3*sp.exp(-3*x3), x5**2, 2*x4*x5, -1, 0, -2*sp.cosh(2*x8)*x9, -sp.sinh(2*x8), 2],
    [sp.cos(x1*x2)*x2, sp.cos(x1*x2)*x1, x9*x7, 0, 6*x5, -sp.exp(-x10+x6) - x8 - 1, x3*x9, -x6, x3*x7, sp.exp(-x10+x6)],
    [1, -1, 1, -1, 1, -1, 1, -1, 1, -1],
    [-x5/((x3+x1)**2), -2*sp.cos(x2**2)*x2, -(x5/((x3+x1)**2)), -2*sp.sin(-x9 + x4), (x3 + x1)**(-1), 0, -2*sp.cos(x7*x10)*sp.sin(x7*x10)*x10, -1, 2*sp.sin(-x9 + x4), -2*sp.cos(x7*x10)*sp.sin(x7*x10)*x7],
    [2*x8, -2*sp.sin(x2), 2*x8, (-x9 + x4)**(-2), sp.cos(x5), x7*sp.exp(-x7*(-x10+x6)), -(x10 - x6)*sp.exp(-x7*(-x10+x6)), 2*x3 + 2*x1, -(x9 - x4)**(-2), -x7*sp.exp(-x7*(-x10+x6))],
    [sp.exp(x1-x4-x9), -3/2*sp.sin(3*x10*x2)*x10, -x6, -sp.exp(x1-x4-x9), 2*(x5/x8), -x3, 0, -(x5**2)/(x8**2), -sp.exp(x1-x4-x9), -3/2*sp.sin(3*x10*x2)*x2],
    [sp.cos(x4), (3*x2**2)*x7, 1, -(x1 - x6)*sp.sin(x4), sp.cos(x10/x5 + x8)*x10*x5**(-2), -sp.cos(x4), x2**3, -sp.cos(x10/x5 + x8), 0, -sp.cos(x10/x5 + x8)*x5**(-1)],
    [2*x5*(x1 - 2*x6), -x7*sp.exp(x2*x7+x10), -2*sp.cos(-x9 + x3), 1.5, (x1 - 2*x6)**2, -4*x5*(x1 - 2*x6), -x2*sp.exp(x2*x7+x10), 0, 2*sp.cos(-x9 + x3), -sp.exp(x2*x7+x10)],
    [-3, -2*x8*x10*x7, 0, sp.exp(x5+x4), sp.exp(x5+x4), -7*x6**(-2), -2*x2*x8*x10, -2*x2*x10*x7, 3, -2*x2*x8*x7],
    [x10, x9, -x8, sp.cos(x4 + x5 + x6)*x7, sp.cos(x4 + x5 + x6)*x7, sp.cos(x4 + x5 + x6)*x7, sp.sin(x4 + x5 + x6), -x3, x2, x1]
])

# функция для вычисления системы уравнений и Якобиана
f = sp.lambdify((x1, x2, x3, x4, x5, x6, x7, x8, x9, x10), equations, 'numpy')
J_func = sp.lambdify((x1, x2, x3, x4, x5, x6, x7, x8, x9, x10), J, 'numpy')

start = time.time()

# метод Ньютона
for i in range(100): # установите своё количество итераций
    J_val = J_func(*x_init)
    f_val = f(*x_init)
    delta = np.linalg.solve(J_val, f_val)
    x_init -= delta
    if np.linalg.norm(delta) < 1e-6: 
        break

print('Решение:', x_init)
print('Количество итераций:', i+1)
print('Время расчета:', time.time() - start)

Решение: [ 0.37306471  0.59384585  1.61029711 -0.56065657 -0.3439438   1.28206619
  0.22015631 -0.22996142  1.2535575  -1.10835269]
Количество итераций: 6
Время расчета: 0.0010004043579101562


b) Решите модифицированным методом Ньютона с матрицей Якоби, посчитанной в начальной точке (используйте LU-разложение для решения СЛАУ!). Посчитайте количество итераций и арифметических операций, затрачиваемых на решение линейных систем (просто поставьте счетчики внутри LU-разложения, считать затраты на матрицу Якоби не надо). Выведите время расчета.

In [8]:
# функция для вычисления системы уравнений и Якобиана
f = sp.lambdify((x1, x2, x3, x4, x5, x6, x7, x8, x9, x10), equations, 'numpy')
J_func = sp.lambdify((x1, x2, x3, x4, x5, x6, x7, x8, x9, x10), J, 'numpy')

start = time.time()

# модифицированный метод Ньютона
J_init = J_func(*x_init) # инициализация матрицы Якоби
n = len(J_init)
L = np.zeros((n, n))
U = np.zeros((n, n))
ops = 0 # счетчик операций

# LU-разложение
for i in range(n):
    for j in range(i, n):
        s = 0
        for k in range(i):
            s += L[i, k] * U[k, j]
            ops += 2
        U[i, j] = J_init[i, j] - s
    L[i, i] = 1
    for j in range(i+1, n):
        s = 0
        for k in range(i):
            s += L[j, k] * U[k, i]
            ops += 2
        L[j, i] = (J_init[j, i] - s) / U[i, i]
        ops += 2

epsilon = 1e-3
for i in range(100): # установите своё количество итераций
    J_val = J_func(*x_init) + epsilon * np.eye(len(x_init)) # добавляем регуляризацию
    L, U, P = linalg.lu(J_val)
    ops = 2/3 * len(x_init)**3
    f_val = f(*x_init)
    y = np.linalg.solve(L, f_val) # решение Ly = f
    delta = np.linalg.solve(U, y) # решение Ux = y
    x_init -= delta
    if np.linalg.norm(delta) < 1e-6: 
        break
        
print('Решение:', x_init)
print('Количество итераций:', i+1)
print('Количество арифметических операций в LU:', ops)
print('Время расчета:', time.time() - start)

Решение: [ 0.37306471  0.59384585  1.61029711 -0.56065657 -0.3439438   1.28206619
  0.22015631 -0.22996142  1.2535575  -1.10835269]
Количество итераций: 1
Количество арифметических операций в LU: 666.6666666666666
Время расчета: 0.0010004043579101562


C:\Users\Nekon\AppData\Local\Temp\ipykernel_2972\4106121429.py:28: RuntimeWarning: divide by zero encountered in double_scalars
  L[j, i] = (J_init[j, i] - s) / U[i, i]
C:\Users\Nekon\AppData\Local\Temp\ipykernel_2972\4106121429.py:28: RuntimeWarning: invalid value encountered in double_scalars
  L[j, i] = (J_init[j, i] - s) / U[i, i]


c) Реализуйте возможность перехода к модифицированному методу Ньютона не сразу. Задайте параметр k начальных пересчетов матрицы. Например, при k=1 будет так же, как и в пункте b). Поэксперементируйте, найдите оптимальное с точки зрения времени счета количество итераций полным методом. Подумайте, можно ли как-то автоматизировать переход на модифицированный метод Ньютона.

В данном случае, необходимо применять модифицированный метод Ньютона только после k итераций. Мы можем достичь этого, просто включив условие в наш цикл. В качестве примера, давайте установим k = 5.

Что касается автоматизации перехода на модифицированный метод Ньютона, мы можем наблюдать за изменением нормы delta и переключиться на модифицированный метод Ньютона, когда изменения становятся достаточно малыми. 

In [9]:
# функция для вычисления системы уравнений и Якобиана
f = sp.lambdify((x1, x2, x3, x4, x5, x6, x7, x8, x9, x10), equations, 'numpy')
J_func = sp.lambdify((x1, x2, x3, x4, x5, x6, x7, x8, x9, x10), J, 'numpy')

start = time.time()

# задаем k
k = 5

start = time.time()

# метод Ньютона с переходом на модифицированный метод после k итераций
for i in range(100): # установите своё количество итераций
    if i < k:
        J_val = J_func(*x_init)
    L, U, P = linalg.lu(J_val)
    ops = 2/3 * len(x_init)**3
    f_val = f(*x_init)
    y = np.linalg.solve(L, f_val) # решение Ly = f
    delta = np.linalg.solve(U, y) # решение Ux = y
    x_init -= delta
    if np.linalg.norm(delta) < 1e-6: 
        break

print('Решение:', x_init)
print('Количество итераций:', i+1)
print('Время расчета:', time.time() - start)

Решение: [ 0.37306471  0.59384585  1.61029711 -0.56065657 -0.3439438   1.28206619
  0.22015631 -0.22996142  1.2535575  -1.10835269]
Количество итераций: 1
Время расчета: 0.0010001659393310547


d) Реализуйте «циклический» вариант, при котором матрица Якоби пересчитывается не каждый раз, а каждые m итераций; посмотрите, как меняется общее число итераций и затраты.

In [10]:
# функция для вычисления системы уравнений и Якобиана
f = sp.lambdify((x1, x2, x3, x4, x5, x6, x7, x8, x9, x10), equations, 'numpy')
J_func = sp.lambdify((x1, x2, x3, x4, x5, x6, x7, x8, x9, x10), J, 'numpy')

start = time.time()

# задаем m
m = 3

start = time.time()

# метод Ньютона с пересчетом матрицы Якоби каждые m итераций
for i in range(100): # установите своё количество итераций
    if i % m == 0:
        J_val = J_func(*x_init)
    L, U, P = linalg.lu(J_val)
    ops = 2/3 * len(x_init)**3
    f_val = f(*x_init)
    y = np.linalg.solve(L, f_val) # решение Ly = f
    delta = np.linalg.solve(U, y) # решение Ux = y
    x_init -= delta
    if np.linalg.norm(delta) < 1e-6: 
        break

print('Решение:', x_init)
print('Количество итераций:', i+1)
print('Время расчета:', time.time() - start)

Решение: [ 0.37306471  0.59384585  1.61029711 -0.56065657 -0.3439438   1.28206619
  0.22015631 -0.22996142  1.2535575  -1.10835269]
Количество итераций: 1
Время расчета: 0.0009999275207519531


e) Объедините все реализации в одну с параметрами m и k. Например m=1, k=infinity будет давать полный метод Ньютона как в пункте a), а m=infinity, k=1 – модифицированный метод b).

In [11]:
def Newton_method(f, J_func, x_init, m=1, k=np.inf, tol=1e-6, max_iter=500):
    """
    f: функция системы уравнений
    J_func: функция, вычисляющая Якобиан
    x_init: начальное приближение
    m: число итераций перед пересчетом матрицы Якоби
    k: число итераций с полным пересчетом матрицы Якоби
    tol: допустимая погрешность
    max_iter: максимальное число итераций
    """
    ops = 0
    x = x_init
    for i in range(max_iter):
        if i % m == 0 or i < k:
            J_val = J_func(*x)
            L, U, P = linalg.lu(J_val)
            ops += 2/3 * len(x)**3
        f_val = f(*x)
        y = linalg.solve(L, P @ f_val)
        ops += len(x)**2
        delta = linalg.solve(U, y)
        ops += len(x)**2
        x -= delta
        if np.linalg.norm(delta) < tol:
            break
    return x, ops, i

#пример:
Newton_method(f, J_func, x_init, m=2, k=5)

(array([ 0.37306471,  0.59384585,  1.61029711, -0.56065657, -0.3439438 ,
         1.28206619,  0.22015631, -0.22996142,  1.2535575 , -1.10835269]),
 866.6666666666666,
 0)

С этой функцией можно легко настроить поведение метода Ньютона, меняя параметры m и k. Например, Newton_method(f, J_func, x_init, m=2, k=5) будет вычислять матрицу Якоби каждые 2 итерации и делать полный пересчет в первые 5 итераций.

f) Возьмите x_5^(0) = -0.2. Посмотрите на результаты работы методов (обратите дополнительное внимание на случай из пункта c) при k<7, k=7 и k>7).

In [12]:
# Функция решения системы методом Ньютона
def newton_method(f, J_func, x_init, m, k):
    x = x_init.copy()
    n = len(x)
    f_val = f(*x)
    for i in range(int(k)):
        if i % int(m) == 0:
            J_val = J_func(*x)
            if np.isnan(J_val).any() or np.isinf(J_val).any():
                raise ValueError("Матрица Якоби содержит значения NaN или inf.")
            L, U, P = linalg.lu(J_val)
        y = linalg.solve(L, f_val)
        delta = linalg.solve(U, y)
        x -= delta
        f_val = f(*x)
        if linalg.norm(delta) < 1e-6:
            break
    return x, i+1

# Начальное приближение
x_init = np.array([0.5, 0.5, 1.5, -1.0, -0.5, 1.5, 0.5, -0.5, 1.5, -1.5])

# Задаем значения m и k
m_values = [1, 7, sys.maxsize]
k_values = [sys.maxsize, 7, 1]

for m, k in zip(m_values, k_values):
    try:
        x_sol, num_iter = newton_method(f, J_func, x_init, m, k)
        print(f"m={m}, k={k}:")
        print("Solution:", x_sol)
        print("Number of iterations:", num_iter)
        print("----------------------------------------")
    except ValueError as e:
        print(f"m={m}, k={k}:")
        print("Error:", str(e))
        print("----------------------------------------")

m=1, k=9223372036854775807:
Error: Матрица Якоби содержит значения NaN или inf.
----------------------------------------
m=7, k=7:
Error: array must not contain infs or NaNs
----------------------------------------
m=9223372036854775807, k=1:
Solution: [ 1.42467662  0.82320375  0.24248079  0.67994414 -1.07663961  9.63883054
  4.82377939 -0.40866004  5.71635023 -2.57098305]
Number of iterations: 1
----------------------------------------


<lambdifygenerated-7>:2: RuntimeWarning: overflow encountered in exp
  return [2*x10 + x4*x5**2 - x6 - x9*sinh(2*x8) + cos(x1*x2) + 2.00043397416539 - exp(-3*x3), x3*x7*x9 + 3*x5**2 - x6*(x8 + 1) - exp(-x10 + x6) + sin(x1*x2) + 10.886272036407, x1 - x10 - x2 + x3 - x4 + x5 - x6 + x7 - x8 + x9 - 3.13619047619048, x5/(x1 + x3) - x8 - sin(x2**2) + cos(x10*x7)**2 + 2*cos(x4 - x9) - 0.17074727050223, 2*x8*(x1 + x3) + sin(x5) + 2*cos(x2) - 0.368589627310128 - exp(-x7*(-x10 + x6)) - 1/(x4 - x9), -x3*x6 + x5**2/x8 + exp(x1 - x4 - x9) + 0.5*cos(3*x10*x2) + 2.04910860167719, x2**3*x7 + x3 + (x1 - x6)*cos(x4) - sin(x10/x5 + x8) - 0.73804300762028, 1.5*x4 + x5*(x1 - 2*x6)**2 - exp(x10 + x2*x7) - 2*sin(x3 - x9) + 3.56683219896938, -3*x1 - 2*x10*x2*x7*x8 + 3*x9 + exp(x4 + x5) - 8.43947345083833 + 7/x6, x1*x10 + x2*x9 - x3*x8 + x7*sin(x4 + x5 + x6) - 0.782380952380952]
<lambdifygenerated-8>:2: RuntimeWarning: overflow encountered in exp
  return array([[-x2*sin(x1*x2), -x1*sin(x1*x2), 3*exp(-3*x3), x

значения m и k можно менять, k можно ставить меньше 7 или больше 7